In [1]:
# Data Manipulation and Preparation
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Evaluation metrics and visualisation
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (mean_squared_error, mean_absolute_error, r2_score, 
                             mean_absolute_percentage_error, confusion_matrix, 
                             accuracy_score, precision_score, recall_score, f1_score)

# Ensure GPU usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

import itertools
import time
import csv

Using device: cuda


In [5]:
# Load cleaned dataset
df = pd.read_csv('BTC_1h_w_Diffs.csv', parse_dates=['open_time'])

# Convert to datetime 
df['open_time'] = pd.to_datetime(df['open_time'])
df['close_time'] = pd.to_datetime(df['close_time'])

In [6]:
df.columns

Index(['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time',
       'quote_vol', 'count', 'buy_base', 'buy_quote', 'open_p', 'high_p',
       'low_p', 'close_p', 'log_open', 'log_high', 'log_low', 'log_close',
       'diff', 'log_diff', 'direction', 'direction_log', 'diff_open',
       'diff_high', 'diff_low', 'diff_close', 'diff_log_open', 'diff_log_high',
       'diff_log_low', 'diff_log_close'],
      dtype='object')

In [8]:
# Helper functions
def create_sequences(data, target, window_size):
    sequences = []
    labels = []
    for i in range(len(data) - window_size):
        seq = data[i:i + window_size]
        label = target[i + window_size]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

def prepare_data(df, target_column, window_size, feature_columns, binary_target=False, gamma=0.001):
    X = df[feature_columns].values
    y = df[target_column].values.reshape(-1, 1)
    
    if binary_target:
        # Convert to binary classification target
        y = np.where((y[1:] / y[:-1] - 1) > gamma, 1, 0)
        y = np.append([0], y) 

    scaler_X = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)
    X_train_seq, y_train_seq = create_sequences(X_train, y_train, window_size)
    X_test_seq, y_test_seq = create_sequences(X_test, y_test, window_size)

    return X_train_seq, X_test_seq, y_train_seq, y_test_seq

In [9]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, lstm_dim, dense_dim, output_dim, num_layers=1, dropout=0.0, activation_function=nn.Tanh):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, lstm_dim, num_layers=num_layers, batch_first=True, dropout=(dropout if num_layers > 1 else 0))
        self.fc1 = nn.Linear(lstm_dim, dense_dim)
        self.activation = activation_function()
        self.fc2 = nn.Linear(dense_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.activation(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x


In [10]:
def train_and_evaluate(df, target_column, feature_columns, params, file_prefix='results', binary_target=False, gamma=0.001, use_early_stopping=True, plot_loss=False):
    result = {} 
    try:
        # Extract parameters
        window_size = params['window_size']
        lstm_dim = params['lstm_dim']
        dense_dim = params['dense_dim']
        num_layers = params['num_layers']
        dropout = params['dropout']
        lr = params['lr']
        batch_size = params['batch_size']
        num_epochs = params['num_epochs']
        optimizer_type = params['optimizer_type']
        patience = params['patience']
        activation_function = params['activation_function']

        # Clear CUDA cache
        torch.cuda.empty_cache()

        # Prepare data
        X_train_seq, X_test_seq, y_train_seq, y_test_seq = prepare_data(df, target_column, window_size, feature_columns, binary_target, gamma)

        # Convert to PyTorch tensors
        X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32).to(device)
        X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32).to(device)
        y_train_tensor = torch.tensor(y_train_seq, dtype=torch.float32).view(-1, 1).to(device)
        y_test_tensor = torch.tensor(y_test_seq, dtype=torch.float32).view(-1, 1).to(device)

        # Create data loaders
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

        # Model parameters
        input_dim = X_train_tensor.shape[2]
        output_dim = 1

        model = LSTMModel(input_dim, lstm_dim, dense_dim, output_dim, num_layers, dropout, activation_function).to(device)

        # Define loss and optimizer
        criterion = nn.BCELoss()  # Binary cross-entropy loss for probabilities
        optimizer = getattr(optim, optimizer_type)(model.parameters(), lr=lr)

        # Training loop
        best_loss = float('inf')
        patience_counter = 0
        start_time = time.time()

        training_losses = []
        validation_losses = []

        for epoch in range(num_epochs):
            model.train()
            epoch_train_loss = 0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                epoch_train_loss += loss.item()

            epoch_train_loss /= len(train_loader)
            training_losses.append(epoch_train_loss)

            model.eval()
            val_loss = 0
            with torch.no_grad():
                for inputs, labels in test_loader:
                    outputs = model(inputs)
                    val_loss += criterion(outputs, labels).item()

            val_loss /= len(test_loader)
            validation_losses.append(val_loss)

        training_time = time.time() - start_time

        # Evaluate the model
        model.eval()
        evaluation_start_time = time.time()
        with torch.no_grad():
            train_predictions = model(X_train_tensor).cpu().numpy()
            test_predictions = model(X_test_tensor).cpu().numpy()

            # Convert probabilities to binary predictions
            train_predictions_binary = (train_predictions > 0.5).astype(int)
            test_predictions_binary = (test_predictions > 0.5).astype(int)

            # Calculate binary classification metrics
            train_acc, train_prec, train_rec, train_f1, train_spec, train_sens = calculate_metrics(y_train_seq, train_predictions_binary)
            test_acc, test_prec, test_rec, test_f1, test_spec, test_sens = calculate_metrics(y_test_seq, test_predictions_binary)

        evaluation_time = time.time() - evaluation_start_time

        result = {
            "window_size": window_size,
            "lstm_dim": lstm_dim,
            "num_layers": num_layers,
            "dense_dim": dense_dim,
            "dropout": dropout,
            "lr": lr,
            "batch_size": batch_size,
            "num_epochs": num_epochs,
            "optimizer_type": optimizer_type,
            "train_acc": train_acc,
            "test_acc": test_acc,
            "train_prec": train_prec,
            "test_prec": test_prec,
            "train_rec": train_rec,
            "test_rec": test_rec,
            "train_f1": train_f1,
            "test_f1": test_f1,
            "train_spec": train_spec,
            "test_spec": test_spec,
            "train_sens": train_sens,
            "test_sens": test_sens,
            "training_time": training_time,
            "evaluation_time": evaluation_time,
            "patience": patience
        }

        # Save result to a CSV file
        with open(f'{file_prefix}_{target_column}.csv', 'a', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=result.keys())
            if f.tell() == 0:
                writer.writeheader()  # Write header only if file is empty
            writer.writerow(result)

        # Print summary of the results
        print(f"Results: Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")
        print(f"Train Prec: {train_prec:.4f}, Test Prec: {test_prec:.4f}")
        print(f"Train Rec: {train_rec:.4f}, Test Rec: {test_rec:.4f}")
        print(f"Train F1: {train_f1:.4f}, Test F1: {test_f1:.4f}")
        print(f"Train Spec: {train_spec:.4f}, Test Spec: {test_spec:.4f}")
        print(f"Train Sens: {train_sens:.4f}, Test Sens: {test_sens:.4f}")
        print(f"Training Time: {training_time:.4f} seconds, Evaluation Time: {evaluation_time:.4f} seconds\n")

        # Plot loss over epochs if specified
        if plot_loss:
            plot_loss_over_epochs(training_losses, validation_losses, f'{file_prefix}_{target_column}', start_epoch=3)

    except Exception as e:
        print(f"An error occurred: {e}")

        result = {
            "window_size": window_size,
            "lstm_dim": lstm_dim,
            "num_layers": num_layers,
            "dense_dim": dense_dim,
            "dropout": dropout,
            "lr": lr,
            "batch_size": batch_size,
            "num_epochs": num_epochs,
            "optimizer_type": optimizer_type,
            "error": str(e)
        }

    return result


In [11]:
# Define parameter space
parameter_space = {
    'window_size': [24],
    'lstm_dim': [60, 150, 200],
    'dense_dim': [60, 128, 200],
    'num_layers': [1],
    'dropout': [0.0, 0.001],
    'lr': [0.0001],
    'num_epochs': [100, 150, 200],
    'batch_size': [64],
    'optimizer_type': ['Adam'],
    'patience': [24],
    'activation_function': [nn.Tanh]
}

# Define target column
target_column = 'close'

# Define feature combination
features = ['high', 'low', 'close', 'volume',
       'quote_vol', 'count', 'buy_base', 'buy_quote', 'high_p', 'low_p', 'close_p']

In [12]:
# Start total timer
start = time.time()

# Calculate total number of models
total_models = len(list(itertools.product(*parameter_space.values())))

# Initialize model counter
model_counter = 1

# Grid search
for params in itertools.product(*parameter_space.values()):
    param_dict = dict(zip(parameter_space.keys(), params))
    print(f"Running model {model_counter}/{total_models} with parameters: {param_dict}\n")
    
    # Run with fixed epochs
    print('Results:\n')
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, file_prefix='results2.0', binary_target=True, gamma=0.001, use_early_stopping=False, plot_loss=True)
    
    # Increment model counter
    model_counter += 1

# Calculate total time
end = time.time()
total_time = end - start

# Print the total time taken
print(f"Total time taken: {total_time} seconds")

Running model 1/54 with parameters: {'window_size': 24, 'lstm_dim': 60, 'dense_dim': 60, 'num_layers': 1, 'dropout': 0.0, 'lr': 0.0001, 'num_epochs': 100, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 24, 'activation_function': <class 'torch.nn.modules.activation.Tanh'>}

Results:

Results: Train Acc: 0.5966, Test Acc: 0.6423
Train Prec: 0.4913, Test Prec: 0.4000
Train Rec: 0.0208, Test Rec: 0.0022
Train F1: 0.0399, Test F1: 0.0044
Train Spec: 0.9855, Test Spec: 0.9981
Train Sens: 0.0208, Test Sens: 0.0022
Training Time: 111.4903 seconds, Evaluation Time: 0.1251 seconds

An error occurred: name 'plot_loss_over_epochs' is not defined
Running model 2/54 with parameters: {'window_size': 24, 'lstm_dim': 60, 'dense_dim': 60, 'num_layers': 1, 'dropout': 0.0, 'lr': 0.0001, 'num_epochs': 150, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 24, 'activation_function': <class 'torch.nn.modules.activation.Tanh'>}

Results:

Results: Train Acc: 0.5969, Test Acc: 0.6423
Train Prec: